In [72]:
import pandas as pd
import os

# WNBA Playoff Prediction

## Data Understanding

Importing datasets

In [73]:
for dataset_name in os.listdir('data'):
    if dataset_name.endswith('.csv'):
        print(dataset_name.split(".")[0]) 

awards_players
players_teams
coaches
players
teams_post
series_post
teams


In [74]:
dataframes = dict()
for dataset_name in os.listdir('data'):
    if dataset_name.endswith('.csv'):
        dataframes[dataset_name.split(".")[0]] = pd.read_csv('data/' + dataset_name) 

## Data Preparation

### Select Data

Eliminating columns with exclusively unique values

In [75]:
for df_name, df in dataframes.items():
    df_clean = df.copy()
    for col in df.columns:
        if len(pd.unique(df[col])) == 1:
            df_clean.drop(col, axis=1, inplace=True)
    dataframes[df_name] = df_clean


### Integrate Data

Merging all dataframes into a single one

In [76]:
data = pd.merge(dataframes['teams_post'], dataframes['teams'], on=['tmID', 'year'])
data = pd.merge(data, dataframes['coaches'], on=['tmID', 'year'])
data = pd.merge(data, dataframes['series_post'], left_on=['tmID', 'year'], right_on=['tmIDWinner', 'year'])
data = pd.merge(data, dataframes['players_teams'], on=['tmID', 'year'], how='inner')
data = pd.merge(data, dataframes['players'], left_on='playerID', right_on='bioID')
data = pd.merge(data, dataframes['awards_players'], on=['playerID'])
